In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('/home/tammy.kojima/Authorship-attribution/')
import feature_extraction as fe
from tqdm import tqdm
import pickle
import os
import ast
from sklearn.model_selection import train_test_split

/home/tammy.kojima/miniconda3/envs/meu_ambiente/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-26 14:08:36,053	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-26 14:08:36,608	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
df_gpt = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt.csv')
df_human = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_human.csv')
df_human = df_human.iloc[:100000]
df_test = pd.concat([df_gpt, df_human], ignore_index=True)
df_test

,text,origin,model
0,"Our rules are the most robust, the president c...",AI,chat gpt
1,...associating outsiders with danger and extre...,AI,chat gpt
2,...alarms were raised in the initial stages of...,AI,chat gpt
3,[The opposition] utilizes every tool to questi...,AI,chat gpt
4,[The governor's] fixation with health protocol...,AI,chat gpt
...,...,...,...
199995,modi took revenge you bcos you are advani team...,human,NaN
199996,really doubt any organisation allowed function...,human,NaN
199997,⃣ already possession this capabilities but thi...,human,NaN
199998,although nasa doesnt have space suits for wome...,human,NaN


In [3]:
# Tokeniza todos os textos primeiro
textos = df_test["text"].fillna("").tolist()
#processed_texts = [fe.preprocess_text(texto) for texto in tqdm(textos, desc="Tokenizando textos")]

In [4]:
#df_prossessed_texts = pd.DataFrame({'text': processed_texts})
#df_prossessed_texts.to_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_tokens_gpt.csv', index=False)
df_prossessed_texts = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_tokens_gpt.csv')

In [5]:
processed_texts = df_prossessed_texts["processed_text"].fillna("").tolist()

In [6]:
lexical_features = [
    fe.extract_lexical_features(textos[i], processed_texts[i])
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_lexical = pd.DataFrame(lexical_features)
df_test = pd.concat([df_test, df_lexical], axis=1)
df_test.head()

Extraindo features estilísticas: 100%|██████████████████████████████████████████████████████████████████████████| 200000/200000 [00:06<00:00, 31720.84it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.151899,158,24,1.0,0.0,0.063291
1,...associating outsiders with danger and extre...,AI,chat gpt,0.150000,180,27,1.0,0.0,0.105556
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.094077,287,27,1.0,0.0,0.097561
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.130435,207,27,1.0,0.0,0.053140
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.212121,132,28,1.0,0.0,0.083333


In [7]:
stylistic_features = [
    fe.extract_stylistic_features(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_stylistic = pd.DataFrame(stylistic_features)
df_test = pd.concat([df_test, df_stylistic], axis=1)
df_test.head()

Extraindo features estilísticas: 100%|██████████████████████████████████████████████████████████████████████████| 200000/200000 [00:02<00:00, 96871.00it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.151899,158,24,1.0,0.0,0.063291,0,0,0.0,0.0,0,0.0,0.0
1,...associating outsiders with danger and extre...,AI,chat gpt,0.150000,180,27,1.0,0.0,0.105556,1,0,0.0,0.0,1,0.0,0.0
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.094077,287,27,1.0,0.0,0.097561,1,0,0.0,0.0,1,0.0,0.0
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.130435,207,27,1.0,0.0,0.053140,0,0,0.0,0.0,0,0.0,0.0
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.212121,132,28,1.0,0.0,0.083333,0,0,0.0,0.0,0,0.0,0.0


In [8]:
structural_features = [
    fe.extract_structural_features(textos[i], processed_texts[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estruturais")
]
df_structural = pd.DataFrame(structural_features)
df_test = pd.concat([df_test, df_structural], axis=1)
df_test.head()

Extraindo features estruturais: 100%|███████████████████████████████████████████████████████████████████████████| 200000/200000 [00:02<00:00, 67618.99it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,stylistic_emoticon_density,structural_has_url,structural_has_mention,structural_has_hashtag,structural_is_retweet,structural_url_density,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.151899,158,24,1.0,0.0,0.063291,0,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,...associating outsiders with danger and extre...,AI,chat gpt,0.150000,180,27,1.0,0.0,0.105556,1,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.094077,287,27,1.0,0.0,0.097561,1,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.130435,207,27,1.0,0.0,0.053140,0,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.212121,132,28,1.0,0.0,0.083333,0,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [9]:
syntactic_features = [
    fe.extract_syntactic_features(tokens)
    for tokens in tqdm(processed_texts, desc="Extraindo features sintáticas")
]
df_syntactic = pd.DataFrame(syntactic_features)
df_test = pd.concat([df_test, df_syntactic], axis=1)
df_test.head()

Extraindo features sintáticas: 100%|███████████████████████████████████████████████████████████████████████████████| 200000/200000 [52:34<00:00, 63.40it/s]


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.151899,158,24,1.0,0.0,0.063291,0,...,0.0,0.0,0.0,0.0,1.684260,2.760874,10.000000,0.0,0.101266,0.329114
1,...associating outsiders with danger and extre...,AI,chat gpt,0.150000,180,27,1.0,0.0,0.105556,1,...,0.0,0.0,0.0,0.0,1.877120,3.137987,12.000000,0.0,0.094444,0.305556
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.094077,287,27,1.0,0.0,0.097561,1,...,0.0,0.0,0.0,0.0,1.863008,3.154021,13.166667,0.0,0.108014,0.341463
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.130435,207,27,1.0,0.0,0.053140,0,...,0.0,0.0,0.0,0.0,1.602931,2.685808,10.818182,0.0,0.101449,0.323671
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.212121,132,28,1.0,0.0,0.083333,0,...,0.0,0.0,0.0,0.0,1.808484,3.074954,15.800000,0.0,0.090909,0.310606


In [15]:
ngrams_df = fe.extract_ngrams_features(processed_texts, max_features=300)
ngrams_df.head()

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
# Resetar os índices de ambos os DataFrames
df_test_reset = df_test.reset_index(drop=True)
ngrams_df_reset = ngrams_df.reset_index(drop=True)
df_combined = pd.concat([df_test_reset, ngrams_df_reset], axis=1)

In [ ]:
# Agora concatenar
df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0}).astype(int)
df_combined.fillna(0)
df_combined.head()

/tmp/ipykernel_3447455/2107212330.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0}).astype(int)


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,"Our rules are the most robust, the president c...",1,chat gpt,0.151899,158,24,1.0,0.0,0.063291,0,...,0.0,0.0,0.0,0.0,1.684260,2.760874,10.000000,0.0,0.101266,0.329114
1,...associating outsiders with danger and extre...,1,chat gpt,0.150000,180,27,1.0,0.0,0.105556,1,...,0.0,0.0,0.0,0.0,1.877120,3.137987,12.000000,0.0,0.094444,0.305556
2,...alarms were raised in the initial stages of...,1,chat gpt,0.094077,287,27,1.0,0.0,0.097561,1,...,0.0,0.0,0.0,0.0,1.863008,3.154021,13.166667,0.0,0.108014,0.341463
3,[The opposition] utilizes every tool to questi...,1,chat gpt,0.130435,207,27,1.0,0.0,0.053140,0,...,0.0,0.0,0.0,0.0,1.602931,2.685808,10.818182,0.0,0.101449,0.323671
4,[The governor's] fixation with health protocol...,1,chat gpt,0.212121,132,28,1.0,0.0,0.083333,0,...,0.0,0.0,0.0,0.0,1.808484,3.074954,15.800000,0.0,0.090909,0.310606


In [14]:
#df_combined.to_csv("/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features.csv", index=False)
df_combined = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features.csv')
df_combined

/tmp/ipykernel_3447455/2488752161.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.read_csv('/home/tammy.kojima/Authorship-attribution/df_pronto/df_gpt_com_features.csv')


,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_repeated_chars,...,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,syntactic_subordinating_conj,syntactic_comma_ratio,syntactic_punct_ratio
0,"Our rules are the most robust, the president c...",1,chat gpt,0.151899,158,24,1.0,0.0,0.063291,0,...,0.0,0.0,0.000000,0.0,1.684260,2.760874,10.000000,0.0,0.101266,0.329114
1,...associating outsiders with danger and extre...,1,chat gpt,0.150000,180,27,1.0,0.0,0.105556,1,...,0.0,0.0,0.000000,0.0,1.877120,3.137987,12.000000,0.0,0.094444,0.305556
2,...alarms were raised in the initial stages of...,1,chat gpt,0.094077,287,27,1.0,0.0,0.097561,1,...,0.0,0.0,0.000000,0.0,1.863008,3.154021,13.166667,0.0,0.108014,0.341463
3,[The opposition] utilizes every tool to questi...,1,chat gpt,0.130435,207,27,1.0,0.0,0.053140,0,...,0.0,0.0,0.000000,0.0,1.602931,2.685808,10.818182,0.0,0.101449,0.323671
4,[The governor's] fixation with health protocol...,1,chat gpt,0.212121,132,28,1.0,0.0,0.083333,0,...,0.0,0.0,0.000000,0.0,1.808484,3.074954,15.800000,0.0,0.090909,0.310606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,modi took revenge you bcos you are advani team...,0,NaN,0.100746,268,27,1.0,0.0,0.108209,0,...,0.0,0.0,0.000000,0.0,1.788780,3.144630,16.444444,0.0,0.108209,0.339552
199996,really doubt any organisation allowed function...,0,NaN,0.177778,135,24,1.0,0.0,0.111111,0,...,0.0,0.0,0.000000,0.0,1.637948,2.735214,10.375000,0.0,0.088889,0.296296
199997,⃣ already possession this capabilities but thi...,0,NaN,0.099010,303,30,1.0,0.0,0.102310,0,...,0.0,0.0,0.000000,0.0,1.891227,3.257440,8.750000,0.0,0.102310,0.320132
199998,although nasa doesnt have space suits for wome...,0,NaN,0.197080,137,27,1.0,0.0,0.094891,0,...,0.0,0.0,0.007299,0.0,1.850921,3.105339,11.500000,0.0,0.116788,0.379562
